## 1. Projekat iz predmeta Mašinsko učenje

Klasifikacija po karakteristikama zemlje

Marija Cvetković 1940

Luka Kocić 2022

In [4]:
import pandas as pd
import numpy as np

In [5]:
print("Hello world!")

Hello world!
